In [1]:
import wandb
import pickle
import time
from src.datahandle import DataLoader
from src.model_mlp import *
#hyper
split= 0.8
batch_size = 32
data_shape = (-1)

directory = '/Users/M/Desktop/Uni_MLIS_car/extras/data/A_training_given/training_data/'
training_folder = 'training_data'
training_labels_file= 'training_norm.csv'

In [2]:
training_object= DataLoader(
        directory,
        training_folder,
        training_labels_file
        )
#train test split
train,test = training_object.LoadModelData_info(
        split = split, 
        batch_size = batch_size)


In [3]:
X,Y = training_object.Load_batch(test[0], data_shape=data_shape)
X = jnp.array(X, dtype=jnp.float32)
Y = jnp.array(Y, dtype=jnp.float32)
params = pickle.load(open('model_pkl', 'rb'))


In [4]:
loss = loss_fn(params,X,Y)

In [5]:
loss

DeviceArray(0.1315821, dtype=float32)

In [48]:
from PIL import Image
from numpy import asarray
def Load_batch_quiz(batch, data_shape=(-1,1)):
    X = []
    Image_order = []
    for instance in batch:
        link = instance[0]
        image = Image.open(link)
        data = asarray(image)
        if data.shape[2] == 4:
            data = data[:,:,:-1]
        if data_shape != 'original':
            data = data.reshape(data_shape)
        image_number = instance[1]
        X.append(data)
        Image_order.append(image_number)
    X = np.array(X)
    Image_order = np.array(Image_order)
    return X,Image_order

In [49]:
quiz_directory = '/Users/M/Desktop/Uni_MLIS_car/extras/data/C_testing_given/test_data/'
quiz_training_folder = 'test_data'
quiz_object= DataLoader(
        quiz_directory,
        quiz_training_folder,
        )
quiz_train = quiz_object.LoadQuizData_info()

In [56]:
X,image_order = Load_batch_quiz(quiz_train, data_shape=data_shape)

In [64]:
prds = np.array(batch_forward(params,X))

In [69]:
prds.shape,image_order.shape

((1020, 2), (1020,))

In [76]:
final_prd = np.column_stack((image_order,prds))

In [78]:
image_order,prds,final_prd

(array([348, 412, 374, ..., 345, 351, 437]),
 array([[0.6347348 , 1.28491   ],
        [0.5784427 , 0.58104503],
        [0.60994077, 0.97673213],
        ...,
        [0.58873916, 0.7157146 ],
        [0.60299397, 0.29155037],
        [0.6174    , 1.069447  ]], dtype=float32),
 array([[3.48000000e+02, 6.34734809e-01, 1.28490996e+00],
        [4.12000000e+02, 5.78442693e-01, 5.81045032e-01],
        [3.74000000e+02, 6.09940767e-01, 9.76732135e-01],
        ...,
        [3.45000000e+02, 5.88739157e-01, 7.15714574e-01],
        [3.51000000e+02, 6.02993965e-01, 2.91550368e-01],
        [4.37000000e+02, 6.17399991e-01, 1.06944704e+00]]))

In [121]:
import pandas as pd
final_ordered = final_prd[final_prd[:, 0].argsort()]
df = pd.DataFrame(final_ordered, columns = ['image_id','angle','speed'])
df = df.astype({'image_id': 'int32'})
df

,image_id,angle,speed
0,1,0.578331,0.582704
1,2,0.607713,0.949040
2,3,0.621875,1.125068
3,4,0.590089,0.730836
4,5,0.608614,0.960247
...,...,...,...
1015,1016,0.617811,1.074555
1016,1017,0.589033,0.718940
1017,1018,0.613803,1.024733
1018,1019,0.602490,0.884124


In [122]:
df.to_csv('submission.csv', index=False,)